In [2]:
import re
from rake_nltk import Rake
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import pytorch_cos_sim # cosine silarity
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
import time

import warnings
warnings.simplefilter('ignore')
from itertools import chain

/home/chirag/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
r = Rake()

In [4]:
def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized


In [5]:
df = pd.read_csv('AnalysisResults/'+'Paytm Money'+'.csv')
reviews = pd.DataFrame()

############################
dfNeg = df[df['Sentiment'] == 'neg']
dfPos = df[df['Sentiment'] == 'pos']


reviews['ReviewText'] = dfNeg.iloc[:,5]
reviews['Cleaned'] = reviews['ReviewText'].apply(clean)

# keyphrases = []
r.extract_keywords_from_sentences(list(reviews['Cleaned']))
keyphrases = r.get_ranked_phrases_with_scores()
final = {}
phraseList = []
for weight, phrase in keyphrases:
    if weight > 40.0:
        final[phrase] = weight
        phraseList.append(phrase)

In [6]:
model = SentenceTransformer('stsb-roberta-large')

In [7]:
embeddings = model.encode(phraseList)

In [8]:
markup = [0]*len(embeddings)

In [9]:
ansDF = pd.DataFrame()
for i in range(len(embeddings)):
    if markup[i] == 0:
        markup[i] = 1
        dic={}
        ansList = [] 
        totalWeight = final[phraseList[i]]
        for j in range(len(embeddings[i:])):
            if pytorch_cos_sim(embeddings[i], embeddings[j]).item() > 0.4 and markup[j]==0:
                markup[j]=1
                ansList.append(phraseList[j])
                totalWeight += final[phraseList[i]]
        dic['KeyPhrase'] = phraseList[i]
        dic['CombinedWeight'] = totalWeight
        dic['SimilarPhrases'] = '|'.join(ansList)
        ansDF = ansDF.append(dic, ignore_index=True)

In [14]:
ansDF.tail()

,KeyPhrase,CombinedWeight,SimilarPhrases
966,cant sign,41.091636,
967,bad security,40.933744,
968,developer act,40.618421,
969,try log,40.216421,
970,spite providing,40.055556,


In [13]:
ansDF.shape

(971, 3)